In [1]:
import json
import os
import time
from typing import Any

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import DataLoader
from torchvision import transforms
import pandas as pd

from sklearn.metrics import f1_score   
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
import warnings
warnings.filterwarnings('always')  # "error", "ignore", "always", "default", "module" or "once"

In [2]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [3]:
from  RoadImageReader import RoadImageReader

In [4]:
from GaborNN import GaborNN

In [5]:
transform = transforms.Compose(
        [
            transforms.ToPILImage(),
            transforms.Resize((256, 256)),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        ]
    )

In [6]:
train_set = RoadImageReader(
        root_dir=os.path.join("C:\\Users\\91701\\OneDrive\\thesiscode\\data", "train"), transform=transform
    )
test_set = RoadImageReader(
        root_dir=os.path.join("C:\\Users\\91701\\OneDrive\\thesiscode\\data", "test"), transform=transform
    )

In [7]:
train = DataLoader(train_set, batch_size=128, shuffle=True, num_workers=7)
test = DataLoader(test_set, batch_size=128, shuffle=True, num_workers=7)

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [9]:
net = GaborNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(net.parameters())
#scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=0.01, max_lr=0.1,cycle_momentum=False)

C:\ProgramData\Anaconda3\lib\site-packages\torch\functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ..\aten\src\ATen\native\TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [10]:
epochs=30

In [11]:
one_layer_gnet_acc_train = []
one_layer_gnet_acc_test = []
time_per_image_train = []
time_per_image_test = []

data_dict = {'MODEL':[],
        'EPOCH':[],
        'TRAINACC':[],
        'TRAINLOSS':[],
        'TESTACC':[],
        'TESTLOSS':[],
        'TIME':[],
        'TRAINPRECISION':[],
        'TRAINRECALL':[],
        'TRAINF1SCORE':[],
        'TESTPRECISION':[],
        'TESTRECALL':[],
        'TESTF1SCORE':[],}

In [12]:
def setDataDict(model,epoch,trainacc,trainloss,testacc,testloss,time,trpre,trre,trf1,tepre,tere,tef1):
    data_dict['MODEL'].append(model)
    data_dict['EPOCH'].append(epoch)
    data_dict['TRAINACC'].append(trainacc)
    data_dict['TRAINLOSS'].append(trainloss)
    data_dict['TESTACC'].append(testacc)
    data_dict['TESTLOSS'].append(testloss)
    data_dict['TIME'].append(time)
    data_dict['TRAINPRECISION'].append(trpre)
    data_dict['TRAINRECALL'].append(trre)
    data_dict['TRAINF1SCORE'].append(trf1)
    data_dict['TESTPRECISION'].append(tepre)
    data_dict['TESTRECALL'].append(tere)
    data_dict['TESTF1SCORE'].append(tef1)

    

In [13]:
def trainingData(net,criterion,optimizer,epochs,modelName):
    for epoch in range(epochs):

        epochTime=time.perf_counter()
        running_loss = 0.0
        correct = 0
        trainrecall=[]
        trainprecision=[]
        testrecall=[]
        testprecision=[]
        label=[]
        net.train()
        start = time.perf_counter()
        for data in train:
            # get the inputs
            inputs, labels = data["image"], data["target"]

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs.to(device))
            loss = criterion(outputs, labels.to(device))
            loss.backward()
            optimizer.step()
            #scheduler.step()
            pred = outputs.max(1, keepdim=True)[1].to("cpu")

            correct += pred.eq(labels.view_as(pred)).sum().item()
            trainprecision.append(precision_score(labels.data, pred))
            trainrecall.append(recall_score(labels.data, pred))
            # print statistics
            running_loss += loss.item()
        finish = time.perf_counter()

        trainPrecisionScore=sum(trainprecision)/len(trainprecision)
        trainRecallScore=sum(trainrecall)/len(trainrecall)
        trainF1score=(2*(trainPrecisionScore*trainRecallScore))/(trainPrecisionScore+trainRecallScore)

        time_per_image_train.append((finish - start) / len(train_set))
        print(
            "[%d] train_acc: %.3f train_loss: %.3f"
            % (epoch + 1, correct / len(train_set), running_loss / len(train_set))
        )
        one_layer_gnet_acc_train.append(correct / len(train_set))
        train_loss=running_loss / len(train_set)
        train_acc=correct / len(train_set)
        running_loss = 0.0
        correct = 0
        start = time.perf_counter()
        with torch.no_grad():
            net.eval()
            for data in test:
                # get the inputs
                inputs, labels = data["image"], data["target"]

                # forward + backward + optimize
                outputs = net(inputs.to(device))
                loss = criterion(outputs, labels.to(device))

                pred = outputs.max(1, keepdim=True)[1].to("cpu")
                correct += pred.eq(labels.view_as(pred)).sum().item()
                testprecision.append(precision_score(labels.data, pred))
                testrecall.append(recall_score(labels.data, pred))
                running_loss += loss.item()
        finish = time.perf_counter()
        testPrecisionScore=sum(testprecision)/len(testprecision)
        testRecallScore=sum(testrecall)/len(testrecall)
        testF1score=(2*(testPrecisionScore*testRecallScore))/(testPrecisionScore+testRecallScore)
        time_per_image_test.append((finish - start) / len(test_set))
        print(
            "[%d] test_acc: %.3f test_loss: %.3f"
            % (epoch + 1, correct / len(test_set), running_loss / len(test_set))
        )
        one_layer_gnet_acc_test.append(correct / len(test_set))
        epochTime=time.perf_counter()-epochTime
        setDataDict(modelName,epoch+1,train_acc,train_loss,correct / len(test_set), running_loss / len(test_set),epochTime,trainPrecisionScore,trainRecallScore,trainF1score,testPrecisionScore,testRecallScore,testF1score)
        torch.save(net, modelName+"\\"+modelName+str(epoch+1)+".pt")
        print("Time Taken in Epoch "+ str(epoch+1)+"  Seconds: "+str(epochTime))

    print("Finished Training")

In [14]:
from GaborNNWithResnet import GaborNNWithResnet

In [15]:
net =GaborNNWithResnet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(net.parameters())

In [16]:
one_layer_gnet_acc_train = []
one_layer_gnet_acc_test = []
time_per_image_train = []
time_per_image_test = []
epochs=30

In [17]:
trainingData(net,criterion,optimizer,epochs,'RESNET18')

[1] train_acc: 0.632 train_loss: 0.005
[1] test_acc: 0.709 test_loss: 0.005
Time Taken in Epoch 1  Seconds: 8441.6435396
[2] train_acc: 0.664 train_loss: 0.005
[2] test_acc: 0.720 test_loss: 0.004
Time Taken in Epoch 2  Seconds: 9804.0646562
[3] train_acc: 0.671 train_loss: 0.005
[3] test_acc: 0.679 test_loss: 0.005
Time Taken in Epoch 3  Seconds: 9766.8223311
[4] train_acc: 0.676 train_loss: 0.005
[4] test_acc: 0.722 test_loss: 0.004
Time Taken in Epoch 4  Seconds: 9851.556368499998
[5] train_acc: 0.675 train_loss: 0.005
[5] test_acc: 0.718 test_loss: 0.004
Time Taken in Epoch 5  Seconds: 9540.671708299997
[6] train_acc: 0.682 train_loss: 0.005
[6] test_acc: 0.698 test_loss: 0.005
Time Taken in Epoch 6  Seconds: 9423.9133669
[7] train_acc: 0.685 train_loss: 0.005
[7] test_acc: 0.723 test_loss: 0.004
Time Taken in Epoch 7  Seconds: 9173.414561599995
[8] train_acc: 0.691 train_loss: 0.005
[8] test_acc: 0.709 test_loss: 0.004
Time Taken in Epoch 8  Seconds: 9122.038816100001
[9] train_ac

In [18]:
torch.save(net, "RESNET18.pt")

In [19]:
df=pd.read_csv("result.csv");

df=pd.concat([df,pd.DataFrame(data_dict)],ignore_index = True)
df.to_csv("result.csv", index=False)
data_dict = {'MODEL':[],
        'EPOCH':[],
        'TRAINACC':[],
        'TRAINLOSS':[],
        'TESTACC':[],
        'TESTLOSS':[],
        'TIME':[],
        'TRAINPRECISION':[],
        'TRAINRECALL':[],
        'TRAINF1SCORE':[],
        'TESTPRECISION':[],
        'TESTRECALL':[],
        'TESTF1SCORE':[],}